In [433]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

In [434]:
data = pd.read_csv("heart.csv")
data.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [435]:
data.isnull().sum()

male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             49
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [436]:
data = data.iloc[:,:]
data.drop("education",axis=1,inplace=True)
data.head()

,male,age,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [437]:
def fillnull(index):
    for i in range(index-1):
        data_column = data.iloc[:,i]
        data_column.dropna(inplace=True)
        data_column_mean = np.mean(data_column)
        data_column = data.iloc[:,i]
        data.iloc[:,i].fillna(data_column_mean,inplace=True)

In [438]:
column_len = data.shape[1]
fillnull(column_len)
data.isnull().sum()
X = data.iloc[:,0:column_len-1]
y = data.iloc[:,column_len-1]

In [439]:
poly = PolynomialFeatures(3)
x = poly.fit_transform(X)
x.shape

(4238, 680)

In [440]:
X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size=0.2,random_state=42)


In [441]:
def sigmoid(z):
    ans = 1/(1+np.exp(-z))
    return ans

In [442]:
def cost(theta,x,y,lnR):
    x = np.matrix(x)
    y = np.matrix(y)
    theta = np.matrix(theta)
    theta = theta.reshape(-1,1)
    first = np.multiply(-y,np.log(sigmoid(x * theta)))
    second = np.multiply((1-y),np.log(1-sigmoid(x * theta)))
    ans = np.sum(first - second)/len(y)
    reg = (lnr/2 * len(y)) * np.sum(np.power(theta[1:theta.shape[0],:],2))
    ans2 = ans + reg
    return ans2

In [443]:
def grad(theta,X,y,lnR):
    X = np.matrix(X)
    y = np.matrix(y)
    theta = np.matrix(theta)
    param = int(theta.ravel().shape[1])
    grad = np.zeros(param)
    theta = theta.reshape(-1,1)
    error = sigmoid(X * theta) - y
    
    for i in range(param):
        term = np.multiply(error,X[:,i])
        
        if (i == 0):
            grad[i] = np.sum(term) / len(X)
        else:
            grad[i] = (np.sum(term) / len(X)) + ((lnR / len(X)) * theta[i])
    
    return grad

In [444]:
x = np.array(X_train)
y = np.array(Y_train)
y = y.reshape(-1,1)
n = x.shape[1]
theta = np.zeros(n,dtype=int)
lnr=0.015


print("x",x.shape,"y",y.shape,"theta",theta.shape)

x (3390, 680) y (3390, 1) theta (680,)


In [445]:
import scipy.optimize as opt
result = opt.fmin_tnc(func=cost,x0=theta,fprime=grad,args=(x,y,lnr))
cost(result[0],x,y,lnr)

c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log
  import sys
c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in multiply
  import sys


0.35868252345338597

In [446]:
x = np.array(x)
X = np.matrix(X_test)
theta_min = np.matrix(result[0])
probability = sigmoid(X * theta_min.T)
list_predict = []
for i in probability:
    if i >=0.5:
        list_predict.append(1)
    else:
        list_predict.append(0)

Y = np.matrix(Y_test)
Y = Y.reshape(-1,1)
counter = 0
for i in range(0,len(list_predict)):
    if Y[i]==list_predict[i]:
        counter = counter + 1
        
accuracy = (counter/len(list_predict))*100
print("Accuracy:{0}".format(accuracy))

Accuracy:84.08018867924528


In [447]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

lr.fit(X_train, Y_train)
pred_y = lr.predict(X_test)
print("Test accuracy: {}".format(lr.score(X_test,Y_test)))

Test accuracy: 0.8443396226415094


c:\users\dell\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [448]:
len(probability)

848

In [449]:
len(Y_test)

848